# Twitter Bot Detection: Classification Modeling


The goal of this project is to use machine learning classification models to detect whether a Twitter user is a bot based on account-level information (e.g. number of followers, number of tweets, etc.). This approach will *not* look at the actual contents of tweets. 

After exploring the data and identifying and engineering some potential features, I'll evaluate several classification models to find the best one for Twitter bot detection.

I'll be searching for models that have balanced scores between precision and recall and strong ROC AUC scores -- while I want the model to accurately label bots as often as possible, I also want to reduce misclassification and not simply label *everything* as a bot.

**Process**
1. Evaluate 'out-of-the-box' models and narrow down candidates to 2-3 models based on cross-validated scoring metrics.
2. Consider class-weight balancing (data is ~70/30 split human/bot)
3. Refine feature selection and tune candidate model parameters
4. Consider ensembling best models with VotingClassifier model
5. Pick best model, and perform full train and test
6. Train best model on full data set and pickle to be used in Flask app

In [ ]:
# Basics
import pandas as pd
import psycopg2 as pg
import numpy as np
import pickle

# Visuals
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from model_evaluation import *

# Models
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

# Model support
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (accuracy_score, classification_report, confusion_matrix, f1_score, auc,
                             plot_confusion_matrix, precision_score, recall_score, roc_auc_score, roc_curve, 
                             precision_recall_curve)

## Data import and set up

Importing data from SQL database and setting up features/interactions from EDA notebook to be used in modeling

In [ ]:
# Postgres info to connect
connection_args = {
    'host': 'localhost',             # Connecting to _local_ version of psql
    'dbname': 'twitter_accounts',    # DB with Twitter info
    'port': 5432                     # port we opened on AWS
}

connection = pg.connect(**connection_args)

In [ ]:
raw_df = pd.read_sql('SELECT * FROM human_bots', connection)


In [ ]:
# drop funny index column
raw_df.drop(columns=['index'], inplace=True)

# Binary classifications for bots and boolean values
raw_df['bot'] = raw_df['account_type'].apply(lambda x: 1 if x == 'bot' else 0)
raw_df['default_profile'] = raw_df['default_profile'].astype(int)
raw_df['default_profile'] = raw_df['default_profile'].astype(int)
raw_df['default_profile_image'] = raw_df['default_profile_image'].astype(int)
raw_df['geo_enabled'] = raw_df['geo_enabled'].astype(int)
raw_df['verified'] = raw_df['verified'].astype(int)

# datetime conversion
raw_df['created_at'] = pd.to_datetime(raw_df['created_at'])
# hour created
raw_df['hour_created'] = pd.to_datetime(raw_df['created_at']).dt.hour

In [ ]:
# usable df setup
df = raw_df[['bot', 'screen_name', 'created_at', 'hour_created', 'verified', 'acct_location', 'geo_enabled', 'lang', 
             'default_profile', 'default_profile_image', 'favourites_count', 'followers_count', 'friends_count', 
             'statuses_count', 'average_tweets_per_day', 'account_age_days']]

In [ ]:
del raw_df

In [ ]:
# Interesting features to look at: 
df['avg_daily_followers'] = np.round((df['followers_count'] / df['account_age_days']), 3)
df['avg_daily_friends'] = np.round((df['followers_count'] / df['account_age_days']), 3)
df['avg_daily_favorites'] = np.round((df['followers_count'] / df['account_age_days']), 3)

# Log transformations for highly skewed data
df['friends_log'] = np.round(np.log(1 + df['friends_count']), 3)
df['followers_log'] = np.round(np.log(1 + df['followers_count']), 3)
df['favs_log'] = np.round(np.log(1 + df['favourites_count']), 3)
df['avg_daily_tweets_log'] = np.round(np.log(1+ df['average_tweets_per_day']), 3)

# Possible interactive features
df['network'] = np.round(df['friends_log'] * df['followers_log'], 3)
df['tweet_to_followers'] = np.round(np.log( 1 + df['statuses_count']) * np.log(1+ df['followers_count']), 3)

# Log-transformed daily acquisition metrics for dist. plots
df['follower_acq_rate'] = np.round(np.log(1 + (df['followers_count'] / df['account_age_days'])), 3)
df['friends_acq_rate'] = np.round(np.log(1 + (df['friends_count'] / df['account_age_days'])), 3)
df['favs_rate'] = np.round(np.log(1 + (df['friends_count'] / df['account_age_days'])), 3)

In [ ]:
df.head()

First round of feature selection - these should be accessible by all modeling types.

In [ ]:
features = ['verified', 
            #'created_at',
            #'hour_created',
            #'lang',
            #'acct_location',
            'geo_enabled', 
            'default_profile', 
            'default_profile_image', 
            'favourites_count', 
            'followers_count', 
            'friends_count', 
            'statuses_count', 
            'average_tweets_per_day',
            #'avg_daily_followers', 
            #'avg_daily_friends',
            #'avg_daily_favorites',
            'network', 
            'tweet_to_followers', 
            'follower_acq_rate', 
            'friends_acq_rate', 
            'favs_rate'
           ]

X = df[features]
y = df['bot']

In [ ]:
X, X_test, y, y_test = train_test_split(X, y, test_size=.3, random_state=1234)

## Basic model evaluation

**Models to be evaluated**
* KNearestNeighbors
* LogisticRegression
* NaiveBayes (Gaussian, Bernoulli, Multinomial)
* DecicionTree
* RandomForest
* XGBoost

In [ ]:
# Models that require scaling: 
knn = KNeighborsClassifier(n_neighbors=10)
lr = LogisticRegression()

# Scaling
scalar = StandardScaler()
scalar.fit(X)
X_train_scaled = scalar.transform(X)

model_list = [knn, lr]
kf = KFold(n_splits=5, shuffle=True, random_state=33)

In [ ]:
multi_model_eval(model_list, X_train_scaled, y, kf)

In [ ]:
# Models that don't require scaling
gnb = GaussianNB()
bnb = BernoulliNB()
mnb = MultinomialNB()
tree = DecisionTreeClassifier()
forest = RandomForestClassifier()
xgb = XGBClassifier()

model_list = [gnb, bnb, mnb, tree, forest, xgb]
kf = KFold(n_splits=3, shuffle=True, random_state=33)

In [ ]:
multi_model_eval(model_list, X, y, kf)

RandomForest and XGBoost seem most promising in terms of balance between Precision and Recall metrics and high ROC AUC scores. 

Next I'll plot out the Precision-Recall and ROC Curves for each under KFolds cross-validation.

In [ ]:
forest = RandomForestClassifier()
kf = KFold(n_splits=5, shuffle=True, random_state=33)

roc_curve_cv(forest, X, y, kf, model_alias='RandomForest')

In [ ]:
precision_recall_cv(forest, X, y, kf, model_alias='RandomForest')

In [ ]:
xgb = XGBClassifier()

kf = KFold(n_splits=5, shuffle=True, random_state=33)

roc_curve_cv(xgb, X, y, kf, model_alias='XGBoost')

In [ ]:
precision_recall_cv(xgb, X, y, kf, model_alias='XGBoost')

XGBoost has a slight edge, but I think it's too close to call with the out of the box models. Next I'll see what kind of effect rebalancing class weights has on the models.

## Class weight balancing

In [ ]:
num_bots = len(df[df['bot'] == 1])
num_humans = len(df[df['bot'] == 0])

print("Number of bots: ", num_bots)
print("Number of humans: ", num_humans)
print(f'Bots / Total %: {(num_bots / len(df))*100:.2f}')

In [ ]:
types = ['Humans', 'Bots']
counts = [num_humans, num_bots]

plt.figure(figsize=(4, 4))
sns.barplot(x = types, y = counts)
plt.title("Number of Entries by Account Type", fontsize=11)
sns.despine();

In [ ]:
# For XGBoost
estimate = num_humans/num_bots
estimate

In [ ]:
forest = RandomForestClassifier(class_weight='balanced')
xgb = XGBClassifier(scale_pos_weight=estimate)

models = [forest, xgb]
kf = KFold(n_splits=5, shuffle=True, random_state=33)

In [ ]:
multi_model_eval(models, X, y, kf)

RandomForest doesn't change that much but XGBoost's Recall score jumped past Precision while remaining fairly balanced.

Let's look at a the curves.

### Cross-Validated Precision-Recall and ROC Curves

#### RandomForest

In [ ]:
roc_curve_cv(forest, X, y, kf, model_alias='RandomForest')

In [ ]:
precision_recall_cv(forest, X, y, kf, model_alias='RandomForest')

#### XGBoost

In [ ]:
roc_curve_cv(xgb, X, y, kf, model_alias='XGBoost')

In [ ]:
precision_recall_cv(xgb, X, y, kf, model_alias='XGBoost')

### Confusion Matrices

Let's take a look at the confusion matrices of each model for a single train/test split.

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.25, random_state=3333)

#### RandomForest

In [ ]:
forest = RandomForestClassifier(class_weight='balanced')

forest.fit(X_train, y_train)
preds = forest.predict(X_val)

In [ ]:
metrics_report(preds, y_val)

#### XGBoost

In [ ]:
xgb = XGBClassifier(scale_pos_weight=estimate)

xgb.fit(X_train, y_train)
preds = xgb.predict(X_val)

In [ ]:
metrics_report(preds, y_val)

### Feature Importance

How does each model use the features? 

In [ ]:
plot_feature_importance(forest, features, model_alias='RandomForest')

In [ ]:
plot_feature_importance(xgb, features, model_alias='XGBoost')

It's actually quite interesting how each model uses the features! Network is important for both, but the rest is pretty scattered. Favourites count seems pretty important in both, so does average tweets per day. I'm surprised that RandomForest doesn't have verified as more important.

What's also interesting to note is that the feature scores in RandomForest are much more even than XGBoost - where network and verified are very high and the rest are much smaller.

If I had to pick one model right now, it'd be XGBoost based on the better balance between precision and recall. I still want to take another look at features, continue to tune model parameters, and possibly ensembling RandomForest and XGBoost to create a super model. 

## Continued Tuning and Feature Selection

In [ ]:
df.head(3)

In [ ]:
features = ['verified', 
            #'created_at',
            'hour_created',
            #'lang',
            #'acct_location',
            'geo_enabled', 
            'default_profile', 
            'default_profile_image', 
            'favourites_count', 
            'followers_count', 
            'friends_count', 
            'statuses_count', 
            'average_tweets_per_day',
            #'avg_daily_followers', 
            #'avg_daily_friends',
            #'avg_daily_favorites',
            'network', 
            'tweet_to_followers', 
            'follower_acq_rate', 
            'friends_acq_rate', 
            #'favs_rate'
           ]

X = df[features]
y = df['bot']

X, X_test, y, y_test = train_test_split(X, y, test_size=.3, random_state=1234)

### Finding the best XGBoost Model

In [ ]:
xgb = XGBClassifier(scale_pos_weight=1.8, 
                    tree_method='hist', 
                    learning_rate=0.1,           
                    eta=0.01,                 
                    max_depth=7,                
                    gamma=0.05,
                    n_estimators=200,
                    colsample_bytree=.8
                   )

model_list = [xgb]

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=33)

multi_model_eval(model_list, X, y, kf)

I think these are pretty solid scores and this is the most balanced version of the model I've been able to come up with thus far. Let's take a look at the curves. 

In [ ]:
roc_curve_cv(xgb, X, y, kf, model_alias='XGBoost')

In [ ]:
precision_recall_cv(xgb, X, y, kf, model_alias='XGBoost')

I like these scores a lot. Still, let's try ensembling RandomForest with XGBoost.

### Ensembling with VotingClassifier

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.3, random_state=1234)

In [ ]:
forest = RandomForestClassifier(class_weight='balanced')
forest = forest.fit(X_train, y_train)

xgb = XGBClassifier(scale_pos_weight=1.8, 
                    tree_method='hist', 
                    learning_rate=0.1,           
                    eta=0.01,                 
                    max_depth=7,                
                    gamma=0.05,
                    n_estimators=200,
                    colsample_bytree=.8
                   )

xgb = xgb.fit(X_train, y_train)

In [ ]:
models = [('forest', forest), ('xgb', xgb)]

voting_classifier = VotingClassifier(estimators=models,
                                     voting='soft',
                                     n_jobs=-1)

voting_classifier = voting_classifier.fit(X_train, y_train)

In [ ]:
voting_classifier_prediction = voting_classifier.predict(X_val)

metrics_report(voting_classifier_prediction, y_val)

Worse! 

It's decided - XGBoost is the winning model.

## Best XGBoost Model: Full Train & Test and Results

Now to train the model on the full training data and test on the hold out set.

In [ ]:
# Full train & test
best_model = XGBClassifier(scale_pos_weight=1.8, 
                    tree_method='hist', 
                    learning_rate=0.1,           
                    eta=0.01,                 
                    max_depth=7,                
                    gamma=0.05,
                    n_estimators=200,
                    colsample_bytree=.8
                   )

In [ ]:
best_model.fit(X, y)

In [ ]:
best_model_prediction = best_model.predict(X_test)

metrics_report(best_model_prediction, y_test)

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, best_model.predict_proba(X_test)[:,1])

model_auc = roc_auc_score(y_test, best_model.predict_proba(X_test)[:,1])

plt.figure(figsize=(4, 4), dpi=100)
plt.plot(fpr, tpr,lw=2, label=f'AUC: {model_auc:.4f}')
plt.plot([0,1],[0,1],c='grey',ls='--', label='Chance Line')
plt.xlim([-0.05,1.05])
plt.ylim([-0.05,1.05])

plt.xlabel('False positive rate', fontsize=10)
plt.ylabel('True positive rate', fontsize=10)
plt.title('ROC Curve for Best XGBoost Model', fontsize=11)
plt.legend(loc='lower right', prop={'size': 8}, frameon=False)
sns.despine()
print(f'ROC AUC score: {model_auc:.4f}')
print("")
plt.show();

In [ ]:
# Prec Recal Curve here

model_precision, model_recall, _ = precision_recall_curve(y_test, best_model.predict_proba(X_test)[:,1])

# plot the precision-recall curves
no_skill = len(y_test[y_test==1]) / len(y_test)

plt.figure(figsize=(4, 4), dpi=100)
plt.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill')
plt.plot(model_recall, model_precision, marker=',', label='XGBoost')
# axis labels
plt.title('Precision-Recall Curve for Best XGBoost Model', fontsize=11)
plt.xlabel('Recall', fontsize=10)
plt.ylabel('Precision', fontsize=10)
# show the legend
plt.legend(loc='upper right', prop={'size': 8}, frameon=False)
# show the plot
sns.despine()

pred = best_model.predict(X_test)
prec_score = precision_score(y_test, pred)
rec_score = recall_score(y_test, pred)

print(f'Precision score: {prec_score:.4f}')
print(f'Rcall score: {rec_score:.4f}')
print("")

These are even better than the CV training scores! 

Literally every metric improved: 

```
Metric     |  CV Score |  Test Score
-----------+-----------+------------
Accuracy:  |   0.8687  |   0.8770
Precision: |   0.8035  |   0.8136
Recall:    |   0.8031  |   0.8091
F1 Score:  |   0.8033  |   0.8144
ROC AUC:   |   0.9310  |   0.9336
```    

In [ ]:
plot_feature_importance(best_model, features, model_alias='XGBoost')

Interestingly enough, the model switched Network and Verification statuses in the feature importance rankings. 

## Train model on the data dataset to use for new predictions

Now to train the model on the full dataset and pickle it for later use to make predictions on live Twitter users using their API.

In [ ]:
features = ['verified', 
            'hour_created',
            'geo_enabled', 
            'default_profile', 
            'default_profile_image', 
            'favourites_count', 
            'followers_count', 
            'friends_count', 
            'statuses_count', 
            'average_tweets_per_day',
            'network', 
            'tweet_to_followers', 
            'follower_acq_rate', 
            'friends_acq_rate', 
           ]

X = df[features]
y = df['bot']

In [ ]:
fully_trained_model = XGBClassifier(scale_pos_weight=1.8, 
                                    tree_method='hist', 
                                    learning_rate=0.1,           
                                    eta=0.01,                 
                                    max_depth=7,                
                                    gamma=0.05,
                                    n_estimators=200,
                                    colsample_bytree=.8
                                   )

fully_trained_model.fit(X, y)

Save as pickle to use later!

In [ ]:
# with open('flask_app/model.pickle', 'wb') as to_write:
#    pickle.dump(fully_trained_model, to_write)